In [ ]:
!pip install neo4j-driver

     |████████████████████████████████| 81kB 6.0MB/s 
  Created wheel for neo4j-driver: filename=neo4j_driver-4.3.1-cp37-none-any.whl size=99417 sha256=41ee61a19160a5ef25310621d114711dfbb536b2691eb94270a4e9f278a75171
  Stored in directory: /root/.cache/pip/wheels/e1/a2/dc/04723963b2def531187153fcc470f92773eb78c97f8fae50f1
Successfully built neo4j-driver


In [ ]:
from neo4j import GraphDatabase, basic_auth
driver = GraphDatabase.driver(
    "bolt://3.93.240.12:7687", 
    auth=basic_auth("neo4j", "fountains-inventory-twins"))
session = driver.session()

# Создание базы

In [ ]:
manuscripts = session.run("LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/dmkrylov/Nikitsky-DB/main/Nikitsky_manuscripts.csv' AS line \
FIELDTERMINATOR ';' \
WITH line WHERE line.manuscript IS NOT NULL \
CREATE (:manuscript {number: toINteger(line.manuscript), excavation: line.excavation, house: line.house, \
start_wide: toINteger(line.start_wide), end_wide: toINteger(line.end_wide), start_narrow: toINteger(line.start_narrow),\
 end_narrow: toInteger(line.end_narrow), total_amount: line.total_amount, letter_type: line.letter_type, high_society: line.high_society})")

In [ ]:
people = session.run("LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/dmkrylov/Nikitsky-DB/main/Nikitsky_people.csv' AS line \
FIELDTERMINATOR ';' \
WITH line  \
CREATE (:person {name: line.name_in_manuscript, id: line.unique_id, number: toINteger(line.manuscript),\
 status: line.status, death_year: toINteger(line.death_year), gender: line.gender})")

In [ ]:
relation_p_m = session.run("OPTIONAL MATCH (n1:person), (n2:manuscript) \
WHERE n1.number = n2.number \
CREATE (n2)-[:Mention]->(n1)")

In [ ]:
relation_p = session.run("LOAD CSV WITH HEADERS FROM  'https://raw.githubusercontent.com/dmkrylov/Nikitsky-DB/main/Nikitsky_relationships.csv' as row \
FIELDTERMINATOR ';' \
WITH row \
MATCH (p1:person) WHERE p1.id = row.Source \
MATCH (p2:person) WHERE p2.id = row.Target \
CALL apoc.create.relationship(p1, row.Relation, {type:row.Class}, p2) YIELD rel \
RETURN rel")

**Если будем добавлять новые табличные данные:**

In [ ]:
def add_manuscripts(table_url):
  cypher_query = f"LOAD CSV WITH HEADERS FROM '{table_url}' AS line FIELDTERMINATOR ';'" + "WITH line WHERE line.manuscript IS NOT NULL CREATE (:manuscript {number: toINteger(line.manuscript), excavation: line.excavation, house: line.house, start_wide: toINteger(line.start_wide), end_wide: toINteger(line.end_wide), start_narrow: toINteger(line.start_narrow), end_narrow: toInteger(line.end_narrow), total_amount: line.total_amount, letter_type: line.letter_type, high_society: line.high_society})"
  results = session.run(cypher_query,parameters={})
  for record in results:
    print(record)

In [ ]:
def add_people(table_url):
  cypher_query = f"LOAD CSV WITH HEADERS FROM '{table_url}' AS line FIELDTERMINATOR ';'" + "WITH line CREATE(:person {name: line.name_in_manuscript, id: line.unique_id, number: toINteger(line.manuscript), status: line.status, death_year: toINteger(line.death_year), gender: line.gender})"
  results = session.run(cypher_query,parameters={})
  for record in results:
    print(record)

In [ ]:
def union():
  cypher_query = "MATCH (n:person) RETURN n.number as number UNION ALL MATCH (n:manuscript) RETURN n.number as number"
  results = session.run(cypher_query,parameters={})
  for record in results:
    print(record)

In [ ]:
def rel_people_manusсripts():
  cypher_query = "OPTIONAL MATCH (n1:person), (n2:manuscript) WHERE n1.number = n2.number CREATE (n2)-[:Mention]->(n1)"
  results = session.run(cypher_query,parameters={})
  for record in results:
    print(record)

In [ ]:
def rel_of_people(table_url):
  cypher_query = f"LOAD CSV WITH HEADERS FROM  '{table_url}' as row FIELDTERMINATOR ';'" + "WITH row MATCH (p1:person) WHERE p1.id = row.Source MATCH (p2:person) WHERE p2.id = row.Target CALL apoc.create.relationship(p1, row.Relation, {type:row.Class}, p2) YIELD rel RETURN rel"
  results = session.run(cypher_query,parameters={})
  for record in results:
    print(record)

**Создаем узлы домов, где были найдены манускрипты**

In [ ]:
nodes = session.run("LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/dmkrylov/Nikitsky-DB/main/Nikitsky_manuscripts.csv' AS line FIELDTERMINATOR ';' WITH line  CREATE (:house {name: line.house, number: toINteger(line.manuscript)})")

In [ ]:
rel = session.run("optional MATCH (n1:house), (n2:manuscript) WITH n1, n2 WHERE n1.number = n2.number CREATE (n2)-[:house]->(n1)")

In [ ]:
nodes2 = session.run("MATCH (n:house) WITH n.name AS name, COLLECT(n) AS nodelist, COUNT(*) AS count WHERE count > 1 CALL apoc.refactor.mergeNodes(nodelist) YIELD node RETURN node")

In [ ]:
delete_number = session.run("MATCH (h:house) REMOVE h.number")

#  Запросы

Все люди, **фигурирующие в грамотах**, напрямую и косвенно связанные с посадниками:

In [ ]:
cypher_query = '''MATCH (p2:person {status:"посадник"})-[*]->(p1:person)
WHERE p1.number <> "null"
RETURN p1'''
results = session.run(cypher_query,
  parameters={})

for record in set(results):
  print(record)

<Record p1=<Node id=52 labels=frozenset({'person'}) properties={'name': 'Самсон Иванов', 'number': 932, 'id': 'samson_932', 'gender': 'm', 'status': 'свидетель?'}>>
<Record p1=<Node id=43 labels=frozenset({'person'}) properties={'name': 'Нос', 'number': 937, 'id': 'nos_937', 'gender': 'm', 'status': 'богатый'}>>
<Record p1=<Node id=42 labels=frozenset({'person'}) properties={'name': 'Михалко', 'number': 937, 'id': 'mihalko_937', 'gender': 'm', 'status': 'приказчик'}>>
<Record p1=<Node id=30 labels=frozenset({'person'}) properties={'name': 'Иван', 'number': 933, 'id': 'ivan_933', 'gender': 'm', 'status': 'посадник'}>>
<Record p1=<Node id=29 labels=frozenset({'person'}) properties={'name': 'игумен', 'number': 933, 'id': 'igumen_933', 'gender': 'm', 'status': 'игумен'}>>


**посмотреть параметр "id" узлов**

In [ ]:
cypher_query = '''
MATCH (n)
RETURN labels(n) AS id
LIMIT 10
'''

results = session.run(cypher_query,
  parameters={})

for record in results:
  print(record['id'])

['person']
['person']
['person']
['person']
['person']
['person']
['person']
['person']
['person']
['person']


**посмотреть атрибуты манускриптов**

In [ ]:
cypher_query = '''MATCH (m:manuscript) RETURN keys(m) limit 1'''

results = session.run(cypher_query,
  parameters={})

for prop in results:
  print(prop)

<Record keys(m)=['start_narrow', 'letter_type', 'house', 'start_wide', 'excavation', 'total_amount', 'number', 'end_narrow', 'high_society', 'end_wide']>


**посмотреть манускрипты**

In [ ]:
cypher_query = '''match (m:manuscript)
return m limit 5'''

results = session.run(cypher_query,
  parameters={})

for record in results:
  print(record)

<Record m=<Node id=11 labels=frozenset({'manuscript'}) properties={'number': 928, 'total_amount': 'участок земли', 'excavation': 'Никитинский', 'end_narrow': 1450, 'high_society': 'NA', 'end_wide': 1450, 'start_narrow': 1430, 'letter_type': 'раздельная грамота', 'house': 'В', 'start_wide': 1410}>>
<Record m=<Node id=12 labels=frozenset({'manuscript'}) properties={'number': 930, 'total_amount': 'NA', 'excavation': 'Никитинский', 'end_narrow': 1410, 'high_society': 'NA', 'end_wide': 1420, 'letter_type': 'церковное', 'start_narrow': 1400, 'house': 'Б', 'start_wide': 1380}>>
<Record m=<Node id=13 labels=frozenset({'manuscript'}) properties={'number': 931, 'total_amount': 'NA', 'excavation': 'Никитинский', 'end_narrow': 1410, 'high_society': 'NA', 'end_wide': 1435, 'letter_type': 'бытовое', 'start_narrow': 1400, 'house': 'В', 'start_wide': 1340}>>
<Record m=<Node id=14 labels=frozenset({'manuscript'}) properties={'number': 932, 'total_amount': 'NA', 'excavation': 'Никитинский', 'end_narrow'

**посмотреть статусы людей, посчитать количество людей с данным статусом, показать в порядке убывания**

In [ ]:
cypher_query = '''
MATCH (p:person)
RETURN DISTINCT p.status, count(*)
ORDER BY COUNT(*) DESC
'''
results = session.run(cypher_query,
parameters={})

for record in results:
  print(record["p.status"], "-", record["count(*)"])

None - 16
посадник - 10
свидетель? - 4
жена - 3
должник - 2
богатый - 2
лично свободный - 1
протоколирующий? - 1
заимодавец - 1
игумен - 1
приказчик - 1


**посмотреть все связи людей**

In [ ]:
cypher_query = '''
MATCH (p:person)
WITH p 
MATCH (p)-[r]->(p2)-[r2]->(p3)
RETURN p, r, p2, r2, p3
'''
results = session.run(cypher_query,
  parameters={})

for record in results:
  print(record['p']['name'], "--", record["r"]["type"], "-->", record['p2']['name'], "--", record["r2"]["type"], "-->", record['p3']['name'])

**посмотреть, кто кого к кому посылает**

In [ ]:
cypher_query = '''
MATCH (p:person)
WITH p 
MATCH path = ()-[:`посылает`]->()-[:`послан к`]->()
RETURN path
'''
results = session.run(cypher_query,
  parameters={})

for record in set(results):
  print(record ['path'])

<Path start=<Node id=36 labels=frozenset({'person'}) properties={'name': 'Юрий', 'number': 937, 'id': 'jurij_937', 'gender': 'm', 'status': 'богатый'}> end=<Node id=43 labels=frozenset({'person'}) properties={'name': 'Нос', 'number': 937, 'id': 'nos_937', 'gender': 'm', 'status': 'богатый'}> size=2>
<Path start=<Node id=49 labels=frozenset({'person'}) properties={'name': 'Парфей', 'number': 948, 'id': 'parfej_948', 'gender': 'm'}> end=<Node id=47 labels=frozenset({'person'}) properties={'name': 'Остафья', 'number': 948, 'id': 'ostafja_948', 'gender': 'm'}> size=2>


**найти кратчайший путь от дома Б к дому В**

In [ ]:
cypher_query = '''MATCH (h:house)
WITH h
MATCH p=shortestPath((h:house {name:'Б'})-[*]-(h2:house {name:'В'}))
RETURN p'''

results = session.run(cypher_query,
  parameters={})

print(results)

In [ ]:
group_manuscripts = session.run('Match (p:people)<-[:]-(m:manuscripts)<-[:]-(h:house) return count(p.id) as frequency m.number as number h.name as name')

ConstraintError: ignored

**посмотреть информацию о человеке**

In [ ]:
def find_people_by_name():
  name = input("Введите имя человека, которого хотите посмотреть:")
  cypher_query = 'MATCH (p:person {name:"'+ name +'"})RETURN p'
  results = session.run(cypher_query,parameters={})
  for record in results:
    print(record)

In [ ]:
find_people_by_name()

Введите имя персонажа, которого хотите посмотреть:Лука
<Record p=<Node id=40 labels=frozenset({'person'}) properties={'name': 'Лука', 'id': 'luka', 'gender': 'm'}>>


**Удаляем узлы и связи**

In [ ]:
delet = session.run('MATCH (h:house) DETACH DELETE h')

In [ ]:
delet_all = session.run("Match (n) Delete n")

In [ ]:
del_rel = session.run('Match (n: people {number: 933})-[r: {type: 'relative'}]-() Delete r')